In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Gaucher_Disease/GSE124283'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Changes in the level of expression of genes involved in the pathogenic mechanisms in rare, inherited metabolic diseases."
!Series_summary	"Inherited metabolic diseases belong to the group of rare diseases (so called ‘orphan diseases’) whose incidence is less than 1: 5 000 live births. Among these diseases the lysosomal storage diseases (LSD) are also distinguished, which are caused by disorders in the lysosomal system resulting from the mutations in the genes coding for lysosomal hydrolases, cofactors, enzymes involved in the posttranslational processing, and proteins present in the lysosomal membrane.  Although about 70 LSD are recognized so far, their pathomechanism is almost unknown. Hitherto existing results of scientific investigations indicate that different cellular pathways and events are involved in the pathogenic processes: autophagy, apoptosis, toxic action of lyso- derivatives of lipid compounds, disordered  Ca2+ ions intracellular hom

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Check if gene expression data is available
is_gene_available = True  # Given the dataset is about gene expression using Illumina technology

# Check for variable availability and data type conversion keys
trait_row = 2 if any('Gaucher' in condition for condition in ['condition: Control', 'condition: NPC-D', 'condition: NPC-N', 'condition: NPC-M', 'condition: NPC?', 'condition: NPC(-)variant?', 'condition: NPC-NBP', 'condition: Gaucher t.1', 'condition: Gaucher', 'condition: NPC(-)-variant', 'condition: N/A', 'condition: NPC(-)']) else None
gender_row = 3 if any('gender' in gender for gender in ['gender: M', 'gender: K', 'gender: N/A']) else None

# Age data seems unavailable in provided sample characteristics dictionary
age_row = None

# Conversion functions
def convert_trait(value):
    val = value.split(':')[-1].strip()
    if val in ['Gaucher t.1', 'Gaucher']:
        return 1
    elif 'Control' in val or 'NPC' in val:
        return 0
    return None

def convert_age(value):
    return None  # Age data is unavailable in the dataset

def convert_gender(value):
    val = value.split(':')[-1].strip()
    if val == 'M':
        return 1
    elif val == 'K':  # Assuming 'K' stands for 'Kobieta' which is Polish for 'Female'
        return 0
    return None

# Save cohort information
save_cohort_info('GSE124283', './output/preprocess/GS2/Gaucher_Disease/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical feature extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Gaucher_Disease', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    # Ensure directory exists before saving
    import os
    os.makedirs('./output/preprocess/GS2/Gaucher_Disease/trait_data', exist_ok=True)
    
    csv_path = './output/preprocess/GS2/Gaucher_Disease/trait_data/GSE124283.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM3526881': [0, 1], 'GSM3526882': [0, 1], 'GSM3526883': [0, 1], 'GSM3526884': [0, 1], 'GSM3526885': [0, 1], 'GSM3526886': [0, 0], 'GSM3526887': [0, 0], 'GSM3526888': [0, 0], 'GSM3526889': [0, 0], 'GSM3526890': [0, 0], 'GSM3526891': [0, 0], 'GSM3526892': [0, 1], 'GSM3526893': [0, 0], 'GSM3526894': [0, 0], 'GSM3526895': [0, 0], 'GSM3526896': [0, 1], 'GSM3526897': [0, 1], 'GSM3526898': [0, 1], 'GSM3526899': [0, 0], 'GSM3526900': [0, 0], 'GSM3526901': [0, 0], 'GSM3526902': [0, 0], 'GSM3526903': [0, 0], 'GSM3526904': [0, 0], 'GSM3526905': [0, 0], 'GSM3526906': [0, 0], 'GSM3526907': [0, 0], 'GSM3526908': [0, 0], 'GSM3526909': [0, 0], 'GSM3526910': [0, 0], 'GSM3526911': [0, 1], 'GSM3526912': [0, 1], 'GSM3526913': [0, 1], 'GSM3526914': [0, 0], 'GSM3526915': [0, 0], 'GSM3526916': [0, 0], 'GSM3526917': [0, 0], 'GSM3526918': [0, 0], 'GSM3526919': [0, 0], 'GSM3526920': [0, 1], 'GSM3526921': [0, 1], 'GSM3526922': [0, 1], 'GSM3526923': [0, 1], 'GSM3526924': [0, 1], 'GSM3526925': [0, 1], 'GSM35269

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['7A5', 'A1BG', 'A1CF', 'A26C3', 'A2BP1', 'A2LD1', 'A2M', 'A2ML1',
       'A3GALT2', 'A4GALT', 'A4GNT', 'AAA1', 'AAAS', 'AACS', 'AACSL', 'AADAC',
       'AADACL1', 'AADACL2', 'AADACL3', 'AADACL4'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = False


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['7A5', 'A1BG', 'A1CF', 'A26C3', 'A2BP1'], 'ORF': ['7A5', 'A1BG', 'A1CF', 'A26C3', 'A2BP1'], 'SPOT_ID': [nan, nan, nan, nan, nan]}


### Step 6: Gene Identifier Mapping

In [7]:
# Since requires_gene_mapping is False, we skip STEP6.
# Therefore, there are no substeps to implement here.


### Step 7: Data Normalization and Merging

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Gaucher_Disease/gene_data/GSE124283.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Gaucher_Disease')

# If the trait is not severely biased, save the cohort information and the merged data.

# 4. Save the cohort information.
save_cohort_info('GSE124283', './preprocessed/Gaucher_Disease/cohort_info.json', True, True, trait_biased, merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Gaucher_Disease/GSE124283.csv'
    unbiased_merged_data.to_csv(csv_path)


For the feature 'Gaucher_Disease', the least common label is '1.0' with 6 occurrences. This represents 4.20% of the dataset.
The distribution of the feature 'Gaucher_Disease' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 66 occurrences. This represents 46.15% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

A new JSON file was created at: ./preprocessed/Gaucher_Disease/cohort_info.json
